# Stacking 

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

root_dir = Path('C:/Users/sinjy/jupyter_notebook/datasets')
data_dir = root_dir / 'kaggle_datasets' / 'Iceberg'
predict_dir = root_dir / 'kaggle_predict'

os.listdir(data_dir)

['statoil-iceberg-classifier-challenge',
 'statoil-iceberg-submissions',
 'submission38-lb01448']

In [2]:
os.listdir(data_dir / 'statoil-iceberg-submissions')

['sub_200_ens_densenet.csv',
 'sub_blend009.csv',
 'sub_fcn.csv',
 'sub_keras_beginner.csv',
 'sub_TF_keras.csv']

In [3]:
os.listdir(data_dir / 'submission38-lb01448')

['submission38.csv', 'submission43.csv']

## pytorch CNN DenseNet Ensemble
## ==> sub_200_ens_densenet.csv

In [4]:
from __future__ import print_function
from __future__ import division

import numpy as np
import pandas as pd

import torch
import sys
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split
from sklearn.utils import shuffle

import logging
import psutil
import os
import scipy.signal
import random
from datetime import datetime
from scipy import signal
import scipy

In [5]:
use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

In [6]:
handler = logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)

In [7]:
def cpuStats():
    print(sys.version) ## python version
    print(psutil.cpu_percent())  ## present cpu utilization rate
    print(psutil.virtual_memory())  ## memory
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2. ** 30
    print('memory GB:', memoryUse)
    
cpuStats()

3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
10.6
svmem(total=16980230144, available=11840733184, percent=30.3, used=5139496960, free=11840733184)
memory GB: 0.19475173950195312


In [8]:
lgr.info("USE CUDA=" + str(use_cuda))

INFO:__main__:USE CUDA=True


In [9]:
seed = 17 * 19
np.random.seed(seed)
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)

### load data

In [10]:
TARGET_VAR = 'target'
BASE_FOLDER = data_dir / 'statoil-iceberg-classifier-challenge'

In [12]:
data = pd.read_json(BASE_FOLDER / 'train.json')

print(data.shape)

(1604, 5)


### shuffle

In [17]:
random.seed(datetime.now())
data = shuffle(data)
data = data.reindex(np.random.permutation(data.index))

In [22]:
data['band_1'] = data['band_1'].apply(lambda x: np.array(x).reshape(75,75))
data['band_2'] = data['band_2'].apply(lambda x: np.array(x).reshape(75,75))

data['inc_angle'] = pd.to_numeric(data['inc_angle'], errors='coerce')

In [24]:
band_1 = np.concatenate([im for im in data['band_1']]).reshape(-1, 75, 75)
band_2 = np.concatenate([im for im in data['band_2']]).reshape(-1, 75, 75)
full_img = np.stack([band_1, band_2], axis=1)

In [26]:
def XnumpyToTensor(x_data_np):
    x_data_np = np.array(x_data_np, dtype=np.float32)
    print(x_data_np.shape)
    print(type(x_data_np))
    
    if use_cuda:
        lgr.info("Using the GPU")
        X_tensor = (torch.from_numpy(x_data_np).cuda())
    else:
        lgr.info("Using the CPU")
        X_tensor = (torch.from_numpy(x_data_np))
        
    print((X_tensor.shape))
    return X_tensor

In [27]:
def YnumpyToTensor(y_data_np):
    y_data_np = y_data_np.reshape((y_data_np.shape[0], 1))
    print(y_data_np.shape)
    print(type(y_data_np))
    
    if use_cuda:
        lgr.info("Using the GPU")
        Y_tensor = (torch.from_numpy(y_data_np)).type(torch.FloatTensor).cuda()
    else:
        lgr.info("Using the CPU")
        Y_tensor = (torch.from_numpy(y_data_np)).type(torch.FloatTensor)
        
    print(type(Y_tensor))
    print(y_data_np.shape)
    print(type(y_data_np))
    return Y_tensor

### dataset

In [28]:
class FullTrainingDataset(torch.utils.data.Dataset):
    def __init__(self, full_ds, offset, length):
        self.full_ds = full_ds
        self.offset = offset
        self.length = length
        assert len(full_ds) >= offset + length, Exception("Parent Dataset not long enough")
        super(FullTrainingDataset, self).__init__()
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, i):
        return self.full_ds[i+self.offset]

In [30]:
validationRatio = 0.11
def trainTestSplit(dataset, val_share=validationRatio):
    val_offset = int(len(dataset) * (1 - val_share))
    print("offset:", str(val_offset))
    return FullTrainingDataset(dataset, 0, val_offset), FullTrainingDataset(dataset, val_offset, len(dataset) - val_offset)

In [33]:
batch_size = 64

from torch.utils.data import TensorDataset, DataLoader
train_imgs = XnumpyToTensor(full_img)
train_targets = YnumpyToTensor(data['is_iceberg'].values)
dset_train = TensorDataset(train_imgs, train_targets)

INFO:__main__:Using the GPU


(1604, 2, 75, 75)
<class 'numpy.ndarray'>


INFO:__main__:Using the GPU


torch.Size([1604, 2, 75, 75])
(1604, 1)
<class 'numpy.ndarray'>
<class 'torch.Tensor'>
(1604, 1)
<class 'numpy.ndarray'>


In [41]:
train_ds, val_ds = trainTestSplit(dset_train)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, 
                                          shuffle=False, num_workers=1)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, 
                                        shuffle=False, num_workers=1)
print(val_loader)
print(train_loader)

offset: 1427


### model

In [42]:
num_epochs = 5
n_channels = 2
total_classes = 1

In [43]:
class Bottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(Bottleneck, self).__init__()
        interChannels = 4 * growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1, 
                               bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3, 
                              padding=1, bias=False)
    
    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

In [44]:
class SingleLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(SingleLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, growthRate, kernel_size=3, padding=1,
                              bias=False)
    
    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = torch.cat((x, out), 1)
        return out

In [45]:
class Transition(nn.Module):
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1, 
                              bias=False)
    
    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out

In [46]:
class DenseNet(nn.Module):
    def __init__(self, growthRate, depth, reduction, nClasses, bottleneck):
        super(DenseNet, self).__init__()
        
        nDenseBlocks = (depth-4) // 3
        if bottleneck:
            nDenseBlocks //= 2
        
        nChannels = 2 * growthRate
        self.conv1 = nn.Conv2d(2, nChannels, kernel_size=3, padding=1, 
                               bias=False)
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, 
                                      bottleneck)
        nChannels += nDenseBlocks * growthRate
        nOutChannels = int(math.floor(nChannels * reduction))
        self.trans1 = Transition(nChannels, nOutChannels)
        
        nChannels = nOutChannels
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, 
                                      bottleneck)
        nChannels += nDenseBlocks * growthRate
        nOutChannels = int(math.floor(nChannels * reduction))
        self.trans2 = Transition(nChannels, nOutChannels)
        
        nChannels = nOutChannels
        self.dense3 = self._make_sense(nChannels, growthRate, nDenseBlocks, 
                                      bottleneck)
        nChannels += nDenseBlocks * growthRate
        
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(128, nClasses)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    
    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.dense3(out)
        out = F.avg_pool2d(F.relu(self.bn1(out)), 8)
        out = out.view(out.size(0), -1)
        out = F.sigmoid(self.fc(out))
        return out